In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, HuberRegressor, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
import numpy as np


from keras.layers import Input, Dense, Dropout
from keras.models import Model
import random
import seaborn as sns
import scipy.stats
import os
import tensorflow as tf

In [ ]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv', index_col='row_id')

train_df = data[~data.isnull().any(axis=1)]
train_df

In [ ]:
print('Number of missing values in training set:',data.isna().sum().sum())


In [ ]:
print('Training cardinalities: \n', data.nunique())

In [ ]:
data.info()


In [ ]:

plt.figure(figsize=(72, 72))
sns.heatmap(train_df.corr(),  annot=True)


insight:


In [ ]:
columns = list(train_df.columns)
columns

In [ ]:
del columns[15:40]
columns = columns[:-15]

In [ ]:
def fit_linear_model(cols):
    global train_df,data
   
    p_test_df = data[data[cols].isna()]

    train_x = train_df.drop(cols,axis=1)
    p_test_df = p_test_df.drop(cols,axis=1)
    p_test_df = p_test_df.fillna(data.mean())

    train_y = train_df.loc[:, cols]


    X_train, X_test, t_train, t_test = train_test_split(train_x, train_y, test_size=0.1, random_state=1)

    preproc = StandardScaler()
    X_tr_f = preproc.fit_transform(X_train)

    preproc = StandardScaler()
    X_tr_f = preproc.fit_transform(X_train)

    model = Ridge()
    model.fit(X_tr_f, t_train)
    X_va_f = preproc.transform(X_test)


    y_va_pred = model.predict(X_va_f)

    mse = mean_squared_error(t_test, y_va_pred)
    print(" %s  MSE: %s " % (cols,mse ))


    test_f = preproc.transform(p_test_df)
    pred = model.predict(test_f)
  
    
    dfff = pd.DataFrame(
        data= pred,
        columns=[cols]
    )


   
    dfff['row_id'] = p_test_df.index
    dfff = dfff.set_index('row_id')
    
    dtq = dfff.stack().reset_index()

    dtq['row-col'] = dtq["row_id"].astype(str) + "-" + dtq["level_1"].astype(str)
    dtq.columns = ['row_id', 'level_1', 'value','row-col']
    dtq1 = dtq.drop(['row_id', 'level_1'], axis=1).loc[:,['row-col','value']]

    return dtq1

   

In [ ]:
outputs_F1_F3 = []

for col in columns:
    output = fit_linear_model(col)
    # break
    outputs_F1_F3.append(output)

In [ ]:
f_4_columns = list(train_df.columns)
f_4_columns = f_4_columns[-15:]
f_4_columns

In [ ]:
df_f4 = data.loc[:,f_4_columns]
df_f4

In [ ]:
def qqq(r):
    comb = []
    for kv in r.items():
        if np.isnan(kv[1]):
            comb.append(kv[0])
    return ','.join(comb)

df_f4['nan_cols'] = df_f4.apply(qqq,axis=1)
df_f4

In [ ]:
train_f4_df = df_f4[df_f4['nan_cols'] == '']
train_f4_df = train_f4_df.drop(['nan_cols'],axis=1)
train_f4_df

In [ ]:
test_f4_df = df_f4[df_f4['nan_cols'] != '']
test_f4_df

In [ ]:
combs = test_f4_df['nan_cols'].value_counts()
# combs = list(combs.items())
combs

In [ ]:
combs = combs.reset_index()
combs

In [ ]:
def fit_keras_model(comb,verbose):
    global train_f4_df,test_f4_df
    k = comb
    
    p_test_df = test_f4_df[test_f4_df['nan_cols'] == k]

    p_test_df = p_test_df.drop(['nan_cols'],axis=1)

    cols = k.split(',')


    train_x = train_f4_df.drop(cols,axis=1)
    p_test_df = p_test_df.drop(cols,axis=1)

    

  
    train_y = train_f4_df.loc[:, cols]

    
    X_train, X_test, t_train, t_test = train_test_split(train_x, train_y, test_size=0.1, random_state=1)

    preproc = StandardScaler()
    X_tr_f = preproc.fit_transform(X_train)


    X_va_f = preproc.transform(X_test)
    validation_data = (X_va_f, t_test)


    np.random.seed(1)
    random.seed(1)
    tf.random.set_seed(1)
    activation = 'swish' 

    inputs = Input(shape=(X_train.shape[1]))
    x = Dense(128, 
              use_bias=True,
              activation=activation,
             )(inputs)
    
    x = Dropout(0.001)(x)
    x = Dense(64, 
              use_bias=True,
              activation=activation,
             )(x)

    x = Dense(32, 
              use_bias=True,
              activation=activation,
             )(x)
    x = Dense(8, 
              use_bias=True,
              activation=activation
             )(x)

    x = Dense(train_y.shape[1],
              use_bias=True,
             )(x)
    model = Model(inputs, x)

    historys = []

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.02),
                      
                      loss='mean_squared_error')

    history = model.fit(X_tr_f, t_train, 
                            validation_data=validation_data, 
                            epochs=60,
                            verbose= verbose,
                            batch_size=16384,
                            shuffle=True)

    loss = history.history['loss'][-1]
    val_loss = history.history['val_loss'][-1]  

    test_f = preproc.transform(p_test_df)
    pred = model.predict(test_f,batch_size=16384, verbose=0)


    dfff = pd.DataFrame(
        data= pred,
        columns=cols
    )
    dfff['row_id'] = p_test_df.index
    dfff = dfff.set_index('row_id')

    dtq = dfff.stack().reset_index()

    dtq['row-col'] = dtq["row_id"].astype(str) + "-" + dtq["level_1"].astype(str)
    dtq.columns = ['row_id', 'level_1', 'value','row-col']
    dtq1 = dtq.drop(['row_id', 'level_1'], axis=1).loc[:,['row-col','value']]
    mes = [i, comb,loss,val_loss]


    return comb, dtq1, loss, val_loss,mes
    
   

   

In [ ]:
outputs_F4 = []
mses_F4 = []
i = 0
total = len(combs)
for q in combs.itertuples():
    comb, output, loss, val_loss,mes = fit_keras_model(q.index, 0)
    i = i + 1
    outputs_F4.append(output)
    mses_F4.append(mes)
    print("%s/%s %s  TRAIN-MES:%s  VAL-MSE: %s " % (str(i), str(total), comb, loss, val_loss ))



In [ ]:
outputsnew = pd.concat([pd.concat(outputs_F1_F3),pd.concat(outputs_F4)], ignore_index=True)
outputsnew

In [ ]:
outputsnew.to_csv('submission.csv', index=False)
outputsnew